# Neural Machine Translation
Machine translation is the task of automatically converting source text in one language to text in another language. In a machine translation task, the input already consists of a sequence of symbols in some language, and the computer program must convert this into a sequence of symbols in another language. Given a sequence of symbols in a source language, there is no one single best translation of that sequence to another language. This is because of the natural ambiguity and flexibility of human language. This makes the challenge of automatic machine translation difficult, perhaps one of the most difficult in artificial intelligence.

# Neural Machine Translation with Attention
To translate a sentence from a language to another one, a human translator reads the sentence part by part, and generates part of translation. ANeural machine translation with attention like a human translator looks at the sentence part by part. To generate each part of translation, the attention mechanism tells a Neural Machine Translation model where it should pay attention to.

This notebook trains a Neural Machine Translation with Attention model that can be used to translate from one language to another, like Farsi to English translation. However, training on real-world translation data can take days of training on GPUs. To evaluate the model, I used it for date translation task. 
We will have the network learn to converts Human Readable dates like '28th of March, 1985 ' to Machine Readable format '1985-03-28'

# Generate Date dataset

Importing necessary packages to generate dataset:

In [1]:
import numpy as np
from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from keras.utils import to_categorical
from utils import *

Using TensorFlow backend.


Let's use Faker package to generate random fake dates.

In [2]:
fake = Faker()
Faker.seed(1988)
random.seed(1988)

In [3]:
# Define format of the data we would like to generate
FORMATS = ['short',
           'medium',
           'long',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'd MMM YYY',
           'd MMMM YYY',
           'dd MMM YYY',
           'd MMM, YYY',
           'd MMMM, YYY',
           'dd, MMM YYY',
           'd MM YY',
           'd MMMM YYY',
           'MMMM d YYY',
           'MMMM d, YYY',
           'dd.MM.YY']


In [4]:
# change this if you want it to work with another language
LOCALE = 'en_US'

The following function generates a fake date using a random format picked from our list FORMATS defined above.

In [5]:
def random_fake_date():
    """
        :returns: tuple containing human readable string, machine readable string, and date object
    """
    dt = fake.date_object()

    try:
        human_readable = format_date(dt, format=random.choice(FORMATS),
                                     locale=LOCALE)  
        human_readable = human_readable.lower().replace(',', '')
        machine_readable = dt.isoformat()

    except AttributeError as e:
        return None, None, None

    return human_readable, machine_readable, dt

Let's see how it is working:

In [7]:
random_fake_date()

('tuesday march 28 1978', '1978-03-28', datetime.date(1978, 3, 28))

## Dataset
Let's run the following cells to generates the dataset of 20k human readable dates and their equivalent, standardized, machine readable dates.

In [8]:
m = 20000
dataset, human_vocab, machine_vocab, inv_machine_vocab = generate_dataset(m)

100%|██████████| 20000/20000 [00:02<00:00, 9587.08it/s]


we've generated:
- `dataset`: a list of tuples of (human readable date, machine readable date).
- `human_vocab`: a python dictionary mapping all characters used in the human readable dates to an integer-valued index.
- `machine_vocab`: a python dictionary mapping all characters used in machine readable dates to an integer-valued index. 
    - **Note**: These indices are not necessarily consistent with `human_vocab`. 
- `inv_machine_vocab`: the inverse dictionary of `machine_vocab`, mapping from indices back to characters. 

Let's take a look at the first 5 entries of dataset. 

In [8]:
dataset[:5]

[('tuesday march 28 1978', '1978-03-28'),
 ('26 january 2007', '2007-01-26'),
 ('29 01 12', '2012-01-29'),
 ('thursday february 7 1980', '1980-02-07'),
 ('february 12 1975', '1975-02-12')]

Let's take a look at our human readable vocabulary:

In [9]:
human_vocab

{' ': 0,
 '.': 1,
 '/': 2,
 '0': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12,
 'a': 13,
 'b': 14,
 'c': 15,
 'd': 16,
 'e': 17,
 'f': 18,
 'g': 19,
 'h': 20,
 'i': 21,
 'j': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'y': 34,
 '<unk>': 35,
 '<pad>': 36}

Let's take a look at our machine readable vocabulary:

In [10]:
machine_vocab

{'-': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10}

Lets take a look at inverse of the machine readable vocabulary

## Preprocessing

Let's preprocess the data and map the raw text data into the index values. 
- We will set Tx=30 
    - We assume Tx is the maximum length of the human readable date.
    - If we get a longer input, we would have to truncate it.
- We will set Ty=10
    - "YYYY-MM-DD" is 10 characters long.

In [9]:
Tx = 30
Ty = 10
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)
print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (20000, 30)
Y.shape: (20000, 10)
Xoh.shape: (20000, 30, 37)
Yoh.shape: (20000, 10, 11)


Let's look at some examples of preprocessed training examples. 

In [10]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source date: 26 january 2007
Target date: 2007-01-26

Source after preprocessing (indices): [ 5  9  0 22 13 25 31 13 28 34  0  5  3  3 10 36 36 36 36 36 36 36 36 36
 36 36 36 36 36 36]
Target after preprocessing (indices): [3 1 1 8 0 1 2 0 3 7]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


# Define Model 
Here are some properties of the model that you may notice: 
- There are two separate LSTMs in this model : pre-attention and post-attention LSTMs on both sides of the attention mechanism. 
- *pre-attention* LSTM is a Bi-directional LSTM. Output sequence (hidden states) of this LSTM is input of the attention mechanism.
     - The pre-attention Bi-LSTM goes through $T_x$ time steps
- *post-attention* LSTM is a LSTM that comes after the attention mechanism.
     - The post-attention LSTM goes through $T_y$ time steps. 
- The attention mechanism computes the context variable $context^{\langle t \rangle}$ for each timestep in the output ($t=1, \ldots, T_y$).

#### one_step_attention
* The inputs to the one_step_attention at time step $t$ are:
    - $[a^{<1>},a^{<2>}, ..., a^{<T_x>}]$: all hidden states of the pre-attention Bi-LSTM.
    - $s^{<t-1>}$: the previous hidden state of the post-attention LSTM 
* one_step_attention computes:
    - $[\alpha^{<t,1>},\alpha^{<t,2>}, ..., \alpha^{<t,T_x>}]$: the attention weights
    - $context^{ \langle t \rangle }$: the context vector:
    
$$context^{<t>} = \sum_{t' = 1}^{T_x} \alpha^{<t,t'>}a^{<t'>}\tag{1}$$ 


Let's load all the packages you will need for definign the model

In [11]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

In [12]:
# Shared Layers of the attention mechanism 
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [13]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a"
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis 
    concat = concatenator([a, s_prev])
    e = densor1(concat)
    energies = densor2(e)
    alphas = activator(energies)
    context = dotor([alphas,a])
    
    return context

In [17]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    outputs = []
    
    a = Bidirectional(LSTM(units=n_a, return_sequences=True))(X)
    
    post_activation_LSTM_cell = LSTM(n_s, return_state = True)
    output_layer = Dense(len(machine_vocab), activation=softmax)
    
    for t in range(Ty):
    
        context = one_step_attention(a, s)
        s, _, c = post_activation_LSTM_cell(inputs=context, initial_state=[s, c])
        out = output_layer(s)
        outputs.append(out)
    
   
    model = Model(inputs=[X,s0,c0], outputs=outputs)
    
    return model

Let's run the following cell to create your model.

In [18]:
n_a = 32
n_s = 64
model = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

## Train the Model

In [20]:
optimizer = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))
model.fit([Xoh, s0, c0], outputs, epochs=20, batch_size=100)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/20
20000/20000 [==============================] - 44s 2ms/step - loss: 12.9768 - dense_4_loss: 2.3566 - dense_4_acc: 0.7376 - dense_4_acc_1: 0.7970 - dense_4_acc_2: 0.4300 - dense_4_acc_3: 0.1443 - dense_4_acc_4: 0.9953 - dense_4_acc_5: 0.5648 - dense_4_acc_6: 0.1884 - dense_4_acc_7: 0.9844 - dense_4_acc_8: 0.3772 - dense_4_acc_9: 0.1590
Epoch 2/20
20000/20000 [==============================] - 27s 1ms/step - loss: 5.8542 - dense_4_loss: 1.3612 - dense_4_acc: 0.9733 - dense_4_acc_1: 0.9745 - dense_4_acc_2: 0.7795 - dense_4_acc_3: 0.5267 - dense_4_acc_4: 1.0000 - dense_4_acc_5: 0.9473 - dense_4_acc_6: 0.6117 - dense_4_acc_7: 0.9996 - dense_4_acc_8: 0.6868 - dense_4_acc_9: 0.5219
Epoch 3/20
20000/20000 [==============================] - 28s 1ms/step - loss: 2.8018 - dense_4_loss: 0.5161 - dense_4_acc: 0.9781 - dense_4_acc_1: 0.9823 - dense_4_acc_2: 0.8873 

## Test the Model

In [22]:
Example_dates = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
for example in Example_dates:
    
    source = string_to_int(example, Tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source)))
    source = source.reshape((1, ) + source.shape)
    prediction = model.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    
    print("source:", example)
    print("output:", ''.join(output)) 

source: 3 May 1979
output: 1979-05-03
source: 5 April 09
output: 2009-04-05
source: 21th of August 2016
output: 2016-08-21
source: Tue 10 Jul 2007
output: 2007-07-10
source: Saturday May 9 2018
output: 2018-05-09
source: March 3 2001
output: 2001-03-03
source: March 3rd 2001
output: 2001-03-31
source: 1 March 2001
output: 2001-03-01
